In [1]:
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, snoop, matplotlib
from pprint import pprint as print

matplotlib.style.use("seaborn-whitegrid")
pd.set_option("display.width", 5000)
pd.set_option("display.max_columns", 60)
plt.rcParams["figure.figsize"] = (15, 10)

%load_ext snoop
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_format = 'svg' 
# show all output not only the last result
from IPython.core.interactiveshell import InteractiveShell
import gdown
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import scrapy, sqlite3

In [ ]:
def select(sql):
    return pd.read_sql(sql, con)

In [2]:
import dropbox
from dropbox.files import WriteMode
from dropbox.files import DownloadArg

from dropbox.exceptions import ApiError, AuthError
access_token = 'sl.BB7FFE2B1H-YNUpHFdN8hfX2ox740v_G7hdvw4qNk5hcRMQ5_JLl7mOQ4fb1U25BU3Rys3qEaw66lTsH01DWG7kt-MlBbdxnZQT4olyRgNIY86gmS-N6H5F7eD2aQflRsPPyYhtZAJ4z'
dbx = dropbox.Dropbox(access_token)

In [16]:
LOCALFILE = 'db.sqlite'
BACKUPPATH = '/db.sqlite'


def backup():
    with open(LOCALFILE, 'rb') as f:
        print("Uploading " + LOCALFILE + " to Dropbox as " + BACKUPPATH +
              "...")
        try:
            dbx.files_upload(f.read(), BACKUPPATH, mode=WriteMode('overwrite'))
        except ApiError as err:
            if (err.error.is_path()
                    and err.error.get_path().reason.is_insufficient_space()):
                sys.exit("ERROR: Cannot back up; insufficient space.")
            elif err.user_message_text:
                print(err.user_message_text)
                sys.exit()
            else:
                print(err)
                sys.exit()


backup()

'Uploading db.sqlite to Dropbox as /db.sqlite...'


In [40]:
dbx.files_download(BACKUPPATH, rev=None);

In [41]:
con = sqlite3.connect('db.sqlite')
# !gdown --id 1Gj0qVU0Acv-HqRjTIEwvkO4TqJYijehR
df = pd.read_csv(
    '/Users/glebsokolov/HeadRepo/DataSets/german_credit_augmented.csv')


In [42]:
sql = '''SELECT
    *
FROM
    german t
WHERE
    t.credit_amount > 1000'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
3,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
4,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
...,...,...,...,...,...,...,...,...,...,...,...,...
879,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
880,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
881,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
882,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


In [46]:
select(sql).dtypes

age                  int64
sex                 object
job                  int64
housing             object
saving_accounts     object
checking_account    object
credit_amount        int64
duration             int64
purpose             object
default              int64
contract_dt         object
client_id            int64
dtype: object

In [32]:
sql = '''DROP TABLE IF EXISTS german_greater_1000;

CREATE TABLE german_greater_1000 AS
SELECT
    *
FROM
    german t
WHERE
    t.credit_amount > 1000'''
con.executescript(sql)


In [33]:
sql = '''select * from german_greater_1000'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
3,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
4,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
...,...,...,...,...,...,...,...,...,...,...,...,...
879,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
880,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
881,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
882,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


In [45]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'])
df.dtypes
df.to_sql('german', con, index=False, if_exists='replace')

age                          int64
sex                         object
job                          int64
housing                     object
saving_accounts             object
checking_account            object
credit_amount                int64
duration                     int64
purpose                     object
default                      int64
contract_dt         datetime64[ns]
client_id                    int64
dtype: object

1000